## 타이타닉 생존율을 위한 데이터 분석 
   - EDA(Exploratory data analysis)기반 데이터 분석
   - EDA는 데이터를 시각화 하고 요약 통계를 통해 이해 함으로써 데이터 구조, 이상치, 패턴 등을 발견하는 활동(과정) 입니다.
   - 데이터 분석 과정의 초기단계에서 중요한 역할을 하며, 데이터에 대한 초기 통찰을 얻어 모델링 작업을 위한 기초를 다져보자. 

##### EDA의 주요목적
  - 데이터 이해
  - 이상치 발견
  - 가설 설정
  - 데이터 변환
##### EDA의 주요기법
  - 기술 통계 : 평균,중앙 값,분산,표준 편차 등 기본 통계량을 계산하여 데이터의 특성을 파악합니다.
  - 데이터 시각화 -seaborn 
  - 상관분석 : 두 변수간의 상관관계를 분석하여 변수간의 관계를 파악합니다.
  - 히트맵(Heatmap) : 상관관계를 시각적으로 표현 합니다.

### 시작하기
- 머신러닝에는 데이터 전처리,회귀,분류,군집화,연관규칙학습,강화학습,자연어처리 등 다양한 분야가 있다.
- 머신러닝 모델을 구축할 때 올바른 데이터로 훈련하기 위해서는 반드시 데이터 전처리 단계부터 시작해야 합니다.

### 머신 러닝 모델 학습을 위한 구현 순서
1.라이브러리 임포트   
2.데이터셋 임포트   
3.결측데이터 처리   
4.범주형 데이터 인코딩 방법   
5.데이터 세트를 훈련세트와 테스트 세트로 나누기   
6.특성 스케일링(필수아님 -필요여부에 따라)   

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm #한글 폰트

공통 코드(로컬) - 한글 폰트 설정  
  - 파일 경로 
    - 상대경로 : 클라우드나 웹서버 공유시 경로 이슈를 해결할 수 있다.
    - 하나 위의 경로는 ../ 로 사용하고 현재 바라보는 경로는 ./를 사용합니다.  
   
예시)  
     - 이 문서의 위치 
        - pythonbasic > EDAProject > XXX.ipynb  
     - 한글 글꼴의 위치 
        - pythonbasic > malgun.ttf 
    

In [ ]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import rc
path = "../malgun.ttf"
font_name = fm.FontProperties(fname=path).get_name()
matplotlib.rc('font', family=font_name)
plt.rc('font', family="Malgun Gothic")

python의 경고 메시지를 무시하는 설정이 있습니다.  
경고 메시지가 출력되지 않게 만들어, 경고로 인한 코드 실행이 방해 받지 않도록 할수 있습니다.

In [ ]:
import warnings
warnings.filterwarnings(action="ignore")

In [ ]:
#훈련데이터 - 모형
df_titanic = pd.read_csv("../data/titanic/train.csv")
#테스트데이터(답안지)
df_test = pd.read_csv("../data/titanic/test.csv")
#1번클립 - EDA 실습,분포,차트, 시각화 분석 , 데이터 전처기, 결측처리...
#2번클립 - 머신러닝(회귀,분류),손실율(기준->판단), 레포팅 작업
# 목표 : Survived 컬럼인 생존여부를 예측하는 프로젝트
submission = pd.read_csv("../data/titanic/gender_submission.csv")

In [ ]:
df_titanic.info()

In [ ]:
df_titanic.describe()

In [ ]:
df_titanic.head(5)

In [ ]:
df_titanic.tail(3)

In [ ]:
df_titanic.isnull()

In [ ]:
df_titanic.isnull().sum()

결측치를 seaborn 활용하여 시각화하기

In [ ]:
import seaborn as sns
#insert here 

### 2-1.Target변수(Survived)

In [ ]:
df_titanic["Survived"].value_counts

In [ ]:
plt.figure(figsize=(8,5))
labels=["사망","생존"]
df_titanic["Survived"].value_counts().plot.pie(explode=[0,0.08]
                                               ,shadow=True
                                               ,autopct="%1.1f%%"
                                               ,labels=labels) # type: ignore

사망 비율은 61.6%, 생존 비율은 38.4%로 사망한 탑승객의 수가 더 많은것으로 확인된다.

In [ ]:
df_titanic.columns

In [ ]:
df_titanic["Survived"].value_counts()

### 2-2 성별(남성:male, 여성:female)
남성의 탑승자 수가 더 많지만 여성의 생존자 수가 더 많다.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].set_title("성별 탑승자 수 \n", size=15)
sns.countplot(x="Sex", data=df_titanic, ax=axes[0])
axes[1].set_title("성별 생존자 수 \n", size=15)
sns.countplot(x="Sex", hue="Survived", data=df_titanic, ax=axes[1])
axes[1].legend(labels = ['사망', '생존'])

### 2-3 티켓 클래스(Pclass) 
  - 티켓 클래스별로 어떤 생존율을 보이고 있는가?  
    - 1등석의 생존율은 약 63%, 2등석의 생존율은 47.3%, 3등석의 생존율은 24.2% 이다  
    - 3등석 탑승객들이 많이 사망한것으로 보인다.

In [ ]:
df_titanic[["Pclass","Survived"]].groupby("Pclass").mean()

위 결과를 시각화해서 살펴봅시다.  
 - 3등석의 탑승객이 가장 많고, 또한 3등석의 탑승객의 사망 인원수가 가장 많습니다.
 - 상대적으로 1등석 탑승객들은 많이 생존한것으로 보인다.
 - 티켓 클래스는 생존여부에 영향을 준 것으로 보인다.

In [ ]:
# 시각화 실습 - 1,2했을때 두개의 차트
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].set_title("클래스별 탑승자 수 \n", size=15)
sns.countplot(x="Pclass", data=df_titanic, ax=axes[0])
axes[1].set_title("클래스별 생존/사망자 수 \n", size=15)
sns.countplot(x="Pclass", hue="Survived", data=df_titanic, ax=axes[1])
axes[1].legend(labels = ['사망', '생존'])

###  2-4 나이 클래스 (Age)  
  - 결측치가 존재하는지 여부
  - 탑승객들의 평균 나이는 30세 입니다.

In [ ]:
df_titanic["Age"].describe()

나이의 분포를 살펴보면 탑승객들의 나이는 주로 20~40세에 분포되어 있습니다.

In [ ]:
plt.figure(figsize=(8,5))
sns.distplot(df_titanic["Age"],bins=25)

### 2-5. 함께 탑승한 가족의 수(SibSp.Parch)
 SibSp = 함께 탑승한 형제자매, 배우자 수 총합 
 Parch = 함께 탑승한 부모, 자녀 수 총합
 SibSp,Parch : 모두 가족 수 이므로 비슷한 경향을 가질것입니다.

In [ ]:
df_titanic["SibSp"].describe()

In [ ]:
df_titanic["Parch"].describe()

In [ ]:
df_titanic[["SibSp","Survived"]].groupby("SibSp").mean()

In [ ]:
df_titanic[["Parch","Survived"]].groupby("Parch").mean()

SibSp - 형제자매, 배우자 총합 
  - 혼자 탑승한 사람보다 3,4명의 SibSp와 탑승한 승객 들의 사망율이 가장 높다
Parch - 부모 자녀 총합  
  - 부모, 자녀없이 혼자 탑승한 사람의 사망율이 가장 높다.

### 2-6. 탑승 항구 (Embarked)
